In [5]:
# import necessary modules
import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta

train_in_file = '../dataset/training/volume(table 6)_training.csv'
train_out_file = '../refer/training_20min_avg_volume.csv'

test_in_file = '../dataset/testing_phase1/volume(table 6)_test1.csv'
test_out_file = '../refer/test_20min_avg_volume.csv'

def avgVolume(in_file, out_file):
    in_file_name = in_file
    out_file_name = out_file

    # Step 1: Load volume data
    fr = open(in_file_name, 'r')
    fr.readline()  # skip the header
    vol_data = fr.readlines()
    fr.close()

    # Step 2: Create a dictionary to caculate and store volume per time window
    volumes = {}  # key: time window value: dictionary
    for i in range(len(vol_data)):
        each_pass = vol_data[i].replace('"', '').split(',')
        tollgate_id = each_pass[1]
        direction = each_pass[2]

        pass_time = each_pass[0]
        pass_time = datetime.strptime(pass_time, "%Y-%m-%d %H:%M:%S")
        time_window_minute = int(math.floor(pass_time.minute / 20) * 20)
        #print pass_time
        start_time_window = datetime(pass_time.year, pass_time.month, pass_time.day,
                                     pass_time.hour, time_window_minute, 0)

        if start_time_window not in volumes:
            volumes[start_time_window] = {}
        if tollgate_id not in volumes[start_time_window]:
            volumes[start_time_window][tollgate_id] = {}
        if direction not in volumes[start_time_window][tollgate_id]:
            volumes[start_time_window][tollgate_id][direction] = 1
        else:
            volumes[start_time_window][tollgate_id][direction] += 1

    # Step 3: format output for tollgate and direction per time window
    fw = open(out_file_name, 'w')
    fw.writelines(','.join(['"tollgate_id"', '"time_window"', '"direction"', '"volume"']) + '\n')
    time_windows = list(volumes.keys())
    time_windows.sort()
    for time_window_start in time_windows:
        time_window_end = time_window_start + timedelta(minutes=20)
        for tollgate_id in volumes[time_window_start]:
            for direction in volumes[time_window_start][tollgate_id]:
                out_line = ','.join(['"' + str(tollgate_id) + '"',                      '"[' + str(time_window_start) + ',' + str(time_window_end) + ')"',
                                 '"' + str(direction) + '"',
                                 '"' + str(volumes[time_window_start][tollgate_id][direction]) + '"',
                               ]) + '\n'
                fw.writelines(out_line)
    fw.close()

def slice_am_pm(df):
    hours = df.time.dt.hour
    df_am = df.loc[(hours < 12)]
    df_pm = df.loc[(hours >= 12)]
    return df_am, df_pm

def slice_time(df):
    df.time = pd.to_datetime(df.time)
    hour = df.time.dt.hour  
    df_prev2h = df.loc[(((hour >= 6) & (hour < 8)) | ((hour >= 15) & (hour < 17)))]
    df_follow2h = df.loc[(((hour >= 8) & (hour < 10)) | ((hour >= 17) & (hour < 19)))]
    return df_prev2h, df_follow2h
    
def train_proc():   
    avgVolume(train_in_file, train_out_file)

def test_proc():   
    avgVolume(test_in_file, test_out_file)
    

In [ ]:
# train_proc()
# test_proc()

In [4]:
train_volume = pd.read_csv(train_out_file)
train_volume_prev2h, train_volume_follow2h = slice_time(train_volume)
train_volume_prev2h = train_volume_prev2h.sort_values(['tollgate_id','direction', 'time_window'])
train_volume_prev2h = train_volume_prev2h.reset_index()
train_volume_prev2h.head(50)

AttributeError: 'DataFrame' object has no attribute 'time'

In [ ]:
test_volume = pd.read_csv(test_out_file)
test_volume_prev2h, test_volume_follow2h = slice_time(test_volume)
test_volume_prev2h = test_volume_prev2h.sort_values(['tollgate_id', 'direction', 'time_window'])
test_volume_prev2h = test_volume_prev2h.reset_index()
test_volume_prev2h.head(20)